This script classifies text and transform it into a more functional version.
It loads and uses the text in the folder "example_text"

In [1]:
import environ
from openai import OpenAI
from pathlib import Path
import pandas as pd
from functools import partial

from utils import get_responses_from_llm

# Import OpenAI key
env = environ.Env()
environ.Env.read_env()
OPENAI_API_KEY = env("OPENAI_API_KEY")

# Set constanst
LLM_MODEL = "gpt-3.5-turbo"
TEMPERATURE = 0
API_CLIENT = OpenAI(api_key=OPENAI_API_KEY)

# Set paths
path_load_data = Path("./example_text", "examples_com_style.csv")
path_save_data = Path("./example_text", "clf_examples_com_style.csv")

/home/daniele/Desktop/Projects/toxicity_detector_playground/.venv_dialogue_ai/lib/python3.10/site-packages/environ/environ.py:639: UserWarning: /tmp/ipykernel_18356/.env doesn't exist - if you're not configuring your environment separately, create one.
  warnings.warn(


In [2]:
# Load data
data = pd.read_csv(path_load_data)
data_melt = pd.melt(data, id_vars=['communication_style'], var_name='user_type', value_name='text')
data_melt

,communication_style,user_type,text
0,Criticism,disfunctional_conversation_parner_1,What’s wrong with you?
1,Criticism,disfunctional_conversation_parner_1,Why would you do that to me?
2,Criticism,disfunctional_conversation_parner_1,You never listen to me
3,Criticism,disfunctional_conversation_parner_1,You always treat me like that.
4,Criticism,disfunctional_conversation_parner_1,"""I can't believe you forgot to pick up the gro..."
...,...,...,...
195,Stonewalling,Unclear,Thank you for initiating this discussion. You ...
196,Stonewalling,Unclear,Let's definitely have an open conversation abo...
197,Stonewalling,Unclear,You're just trying to avoid taking responsibil...
198,Stonewalling,Unclear,"I agree, addressing tension in a calm and logi..."


# Classify the text

In [3]:
# Set all parameters but "user_text"
partial_get_responses_from_llm = partial(
    get_responses_from_llm,
    api_client=API_CLIENT,
    llm_model=LLM_MODEL,
    temperature=TEMPERATURE
)

In [4]:
clf_text = []
edited_text = []

N = 10

for i in range(N):
    clf_text_tmp, edited_text_tmp, _ = partial_get_responses_from_llm(user_text=data_melt.text[i])
    clf_text.append(clf_text_tmp)
    edited_text.append(edited_text_tmp)
    if (i % 5) == 0:
        print(f"Done index {i} of {N}")

Done index 0 of 10
Done index 5 of 10


In [5]:
# Remove leading newline characters and replace "\n    " with "\n"
clf_text = [s.lstrip() for s in clf_text]
clf_text = [s.replace('\n    ', '\n') for s in clf_text]
edited_text = [s.lstrip() for s in edited_text]
edited_text = [s.replace('\n    ', '\n') for s in edited_text]

# Add columns with "clf_text" and "edited_text"
data_out = data_melt.iloc[:N,:].copy()
data_out["clf_text"] = clf_text
data_out["edited_text"] = edited_text

# Save to csv
# data_out.to_csv(path_save_data)

In [6]:
data_out

,communication_style,user_type,text,clf_text,edited_text
0,Criticism,disfunctional_conversation_parner_1,What’s wrong with you?,Category: criticism\nText: What’s wrong with you?,Category: criticism\nFunctional Text: Let's ta...
1,Criticism,disfunctional_conversation_parner_1,Why would you do that to me?,Category: criticism\nText: Why would you do th...,Category: criticism\nFunctional Text: Can you ...
2,Criticism,disfunctional_conversation_parner_1,You never listen to me,Category: criticism\nText: You never listen to me,Category: criticism\nFunctional Text: Let's wo...
3,Criticism,disfunctional_conversation_parner_1,You always treat me like that.,Category: criticism\nText: You always treat me...,Category: criticism\nFunctional Text: Let's di...
4,Criticism,disfunctional_conversation_parner_1,"""I can't believe you forgot to pick up the gro...",Category: criticism\nText: I can't believe you...,Category: criticism\nFunctional Text: Let's wo...
5,Criticism,disfunctional_conversation_parner_1,It's always excuses with you. You never take r...,Category: defensiveness\nText: It's always exc...,Category: defensiveness\nFunctional Text: Let'...
6,Criticism,disfunctional_conversation_parner_1,You're just not trying hard enough,Category: criticism\nText: You're just not try...,Category: criticism\nFunctional Text: Let's wo...
7,Criticism,disfunctional_conversation_parner_1,You're too messy,Category: criticism\nText: You're too messy,Category: criticism\nFunctional Text: Let's wo...
8,Criticism,disfunctional_conversation_parner_1,You're too focused on yourself.,Category: criticism\nText: You're too focused ...,Category: criticism\nFunctional Text: Try to c...
9,Criticism,disfunctional_conversation_parner_1,You always ruin things,Category: criticism\nText: You always ruin things,Category: criticism\nFunctional Text: Let's di...
